In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from evaluate import load as load_metric

def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak. 
    This is a workaround to avoid storing too many tensors that are not needed. 
    Taken from https://discuss.huggingface.co/t/cuda-out-of-memory-when-using-trainer-with-compute-metrics/2941/22
    """
    
    # Handle tuple logits (happens when the model is trained using LoRA)
    if isinstance(logits, tuple):
        logits = logits[1]         

    # Proceed with argmax
    pred_ids = torch.argmax(logits, dim=-1)

    return pred_ids
def preprocess_data(examples, tokenizer, max_input_length, max_output_length):
    """Preprocess the dataset by tokenizing inputs and targets."""
    pre_prompt = "Summarize the following Arabic text: "
    inputs = [pre_prompt + text for text in examples['text']]  # Replace with your input column name
    targets = examples['summary']  # Replace with your target column name

    # Tokenize inputs and targets with truncation and padding
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=max_output_length, truncation=True, padding="max_length").input_ids

    model_inputs["labels"] = labels
    return model_inputs

def compute_metrics(eval_pred):
    """Compute ROUGE score for evaluation."""
    rouge = load_metric("rouge")
    predictions, references = eval_pred
    
    # Convert any numpy int64/int32 to regular Python int to avoid overflow
    predictions = [[int(token) for token in pred] for pred in predictions]
    references = [[int(token) for token in ref] for ref in references]

    # Decode predictions and references
    decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions]
    decoded_refs = [tokenizer.decode(ref, skip_special_tokens=True) for ref in references]
    
    print(f'decoded_preds[0]: {decoded_preds[0]}')
    print(f'decoded_refs[0]: {decoded_refs[0]}')

    # Compute ROUGE scores
    result = rouge.compute(predictions=decoded_preds, references=decoded_refs, use_stemmer=True)

    # Extract and return key metrics
    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"]
    }

# Configuration
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
DATASET_NAME = "BounharAbdelaziz/Arabic-Synthetic-Summarization-Dataset-Filtered"  # Replace with the dataset name or path
TEXT_COLUMN = "text"  # Input text column name
SUMMARY_COLUMN = "summary"  # Output summary column name
OUTPUT_DIR = "./qwen-arabic-summarization"
MAX_INPUT_LENGTH = 2048
MAX_OUTPUT_LENGTH = 2048
BATCH_SIZE = 1
EPOCHS = 1
LR = 2e-5

# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Load dataset
dataset = load_dataset(DATASET_NAME)
dataset['validation'] = dataset['validation'].select(range(5))

# Preprocess dataset
tokenized_dataset = dataset.map(
    lambda x: preprocess_data(x, tokenizer, MAX_INPUT_LENGTH, MAX_OUTPUT_LENGTH),
    batched=True,
    remove_columns=dataset["train"].column_names
)

# Define training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    evaluation_strategy="steps",
    save_strategy="steps",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=30,
    eval_steps=30,
    save_total_limit=1,
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available(),
    gradient_checkpointing=True,
    # report_to=["tensorboard"],
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics, # avoids OOM in eval
)

# Train the model
trainer.train()

# Save the final model
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

In [ ]:
decoded_preds[0]: .clearize:

 key text text into 
 العديد من الأواقب التي يمكنطلبى علىعاسة ال فيوف م مة من الهد الآخر. العلاقات الجنسرمة. هذه أناء العلاقاتقة والواصل بين العلاقات، خلال استخدامفاعل عنماع،تف فيتقترام الآكارك..اق التروج للعاًماع بالبعبةككما.. يمكن كنتاولتعك أنظهبار عنك ليس مجردريقة الوحيدة التي ت فيها استخدامقرب،، ف يكون إلى إلى تفكير في إذا كان هذه ال س الس تريدهم.اً إذا الجهمتع أن تتعلمب في شخصريكك فيكون ب بزيات م الجنسنسية، هذا هذاعب علىسي تعبية أونلا لائعة لظه النفسريكك ووفيف الضوتره. أنة إلىمارسة الجنس. يمكنحرص مع فيجانبك شكما و وماد إلى بعضكسائلائ لسق. بعضصة أو إذا أيضا أيضا ت بعضتب التيطرقوب. إذاساعد هذادركنسسما على نوعجة الاحةتكما إذا إذا كنت ترببي، الج العلاقةور، فهول ت تبدأقيدي عنليلاً إذا أيضالوس بعالاً جاك أو. بعضنايكدميكما.. إذا أيضا أيضادمة الك الكصص الممانسية أوكيضكما. خلال تجت اانتباه.ا عن الجنس.الغمما تتب تتمكنمن).). بعض قغككصةص). إذاحرصوا بعا علىنا إلى بعضصومات ابدأا عن أات مغنية التينعلبيهاقرب منكريكك.ني بتجكلوكما. ا، تنسلي ت من ذلكاههة بين شيءك.. تنتضانككما. اجعلسب فيملاعات الرياي وبعضكما.. تطلبي بعضعضا..عاقا. أجل.أدث عن عن الأالئلة حولفتعة لللاريكك"). ابدأا م ط واحد منعااطلين.ابست على جانبكما علىكابة على الأفس..احولا و اابتعل الأفسات.جرد أنانتهاء منكية الأود. ااروا بعض الأانتوه.فيفة.مشع من الأصفائ.لا الألم.. ا كنت تلقة بشأن تدي الردم، جنسسة،قيقيمةية، فع نفسسائد علىكما ش.مما الاقات ل الأتين. ا كنت تكنثر هذهسائد جبطانيات،قيفةتهاما، ف تأس. استخدامحبالهريكك عن كان يرغبوي أن ذلك آخر.خدمعي ذلكاومةته. إذا الجنس بطب الذي تكون فيه فيه. جانبضك إذانتي بعض الأشيواء والطانيتين.زرقة. اجعلان على مكانك.ابدي علىلمما. فينتئ في الأصص الرو ا ت هذالوسماقرب من بعضريكك إلى تعوبة في التاومة بعضج الأم.سب من  النتضانه إذا كنتاجت أنك تشعرك في تطوروز الحودك،،ثدة،هماقا، فحتعدي ق هذاريكك. تنتفي تلوس م مكان آخر. إذا إذا كنت تلقة بشأن تربة، شريكك،طرقناد عن عن، امكنكمافكع باستتك في أناح إلى نحو المثال إلىقولذ أن " "أروا معذيمني". أنلوس هنا". لكنعني نست في طرضكة".". أو "أعر بالحض التغ في أست بعضراحة فينجلس إلىطب". أو تساعدك هذاروج من مكانماكن الأخرى أو تسر الوتبكماية. إذا كنتحدثت ركما رور،درجةكعا للست بالر،سبالها، قد تأس من تحدث عنطلب "ك ت ترغبرين بالراحة. هذهمتار. إذا أن تبقىترم شريكك أنغباتك الجنس أساس. إذا إذا كنتاولتريكك إخاعك بأنضرورة حددود، تضعتها لك عامعمر، ف تحتاجين إلى التفكير في كيفيةغار العلاقةلاقتك.ه. إذا الرغم من أن قد الصمكن أن تجاعلك شريكك عن رغيه في واحدة، الأين،آخر، إلا أن من ي أن تصبح ش بالقلق حهدرك. إذاHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHuman
decoded_refs[0]: في العلاقات السليمة، يمكنك بناء الثقة والتقارب من خلال التحدث والاستماع والمشاركة واحترام أفكاركما. يمكن استخدام نشاطات غير جنسية مثل اللعب بألعاب لوحية أو القراءة معا. يمكنك أيضا إجراء استطلاعات رأي أو مشاهدة فيلم مع وسائد وأضواء. إذا شعرت بالقلق، ابتعدي قليلا أو تقدمي لسببًا لنقل المكان. يجب أن يحترم شريكك رغباتك ويشعرك بالراحة.
decoded_preds[0]: .clearize:

 key text text into 
 العديد من الأواقب التي يمكنطلبى علىعاسة ال فيوف م مة من الهد الآخر. العلاقات الجنسرمة. هذه أناء العلاقاتقة والواصل بين العلاقات، خلال استخدامفاعل عنماع،تف فيتقترام الآكارك..اق التروج للعاًماع بالبعبةككما.. يمكن كنتاولتعك أنظهبار عنك ليس مجردريقة الوحيدة التي ت فيها استخدامقرب،، ف يكون إلى إلى تفكير في إذا كان هذه ال س الس تريدهم.اً إذا الجهمتع أن تتعلمب في شخصريكك فيكون ب بزيات م الجنسنسية، هذا هذاعب علىسي تعبية أونلا لائعة لظه النفسريكك ووفيف الضوتره. أنة إلىمارسة الجنس. يمكنحرص مع فيجانبك شكما و وماد إلى بعضكسائلائ لسق. بعضصة أو إذا أيضا أيضا ت بعضتب التيطرقوب. إذاساعد هذادركنسسما على نوعجة الاحةتكما إذا إذا كنت ترببي، الج العلاقةور، فهول ت تبدأقيدي عنليلاً إذا أيضالوس بعالاً جاك أو. بعضنايكدميكما.. إذا أيضا أيضادمة الك الكصص الممانسية أوكيضكما. خلال تجت اانتباه.ا عن الجنس.الغمما تتب تتمكنمن).). بعض قغككصةص). إذاحرصوا بعا علىنا إلى بعضصومات ابدأا عن أات مغنية التينعلبيهاقرب منكريكك.ني بتجكلوكما. ا، تنسلي ت من ذلكاههة بين شيءك.. تنتضانككما. اجعلسب فيملاعات الرياي وبعضكما.. تطلبي بعضعضا..عاقا. أجل.أدث عن عن الأالئلة حولفتعة لللاريكك"). ابدأا م ط واحد منعااطلين.ابست على جانبكما علىكابة على الأفس..احولا و اابتعل الأفسات.جرد أنانتهاء منكية الأود. ااروا بعض الأانتوه.فيفة.مشع من الأصفائ.لا الألم.. ا كنت تلقة بشأن تدي الردم، جنسسة،قيقيمةية، فع نفسسائد علىكما ش.مما الاقات ل الأتين. ا كنت تكنثر هذهسائد جبطانيات،قيفةتهاما، ف تأس. استخدامحبالهريكك عن كان يرغبوي أن ذلك آخر.خدمعي ذلكاومةته. إذا الجنس بطب الذي تكون فيه فيه. جانبضك إذانتي بعض الأشيواء والطانيتين.زرقة. اجعلان على مكانك.ابدي علىلمما. فينتئ في الأصص الرو ا ت هذالوسماقرب من بعضريكك إلى تعوبة في التاومة بعضج الأم.سب من  النتضانه إذا كنتاجت أنك تشعرك في تطوروز الحودك،،ثدة،هماقا، فحتعدي ق هذاريكك. تنتفي تلوس م مكان آخر. إذا إذا كنت تلقة بشأن تربة، شريكك،طرقناد عن عن، امكنكمافكع باستتك في أناح إلى نحو المثال إلىقولذ أن " "أروا معذيمني". أنلوس هنا". لكنعني نست في طرضكة".". أو "أعر بالحض التغ في أست بعضراحة فينجلس إلىطب". أو تساعدك هذاروج من مكانماكن الأخرى أو تسر الوتبكماية. إذا كنتحدثت ركما رور،درجةكعا للست بالر،سبالها، قد تأس من تحدث عنطلب "ك ت ترغبرين بالراحة. هذهمتار. إذا أن تبقىترم شريكك أنغباتك الجنس أساس. إذا إذا كنتاولتريكك إخاعك بأنضرورة حددود، تضعتها لك عامعمر، ف تحتاجين إلى التفكير في كيفيةغار العلاقةلاقتك.ه. إذا الرغم من أن قد الصمكن أن تجاعلك شريكك عن رغيه في واحدة، الأين،آخر، إلا أن من ي أن تصبح ش بالقلق حهدرك. إذاHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHuman
decoded_refs[0]: في العلاقات السليمة، يمكنك بناء الثقة والتقارب من خلال التحدث والاستماع والمشاركة واحترام أفكاركما. يمكن استخدام نشاطات غير جنسية مثل اللعب بألعاب لوحية أو القراءة معا. يمكنك أيضا إجراء استطلاعات رأي أو مشاهدة فيلم مع وسائد وأضواء. إذا شعرت بالقلق، ابتعدي قليلا أو تقدمي لسببًا لنقل المكان. يجب أن يحترم شريكك رغباتك ويشعرك بالراحة.
decoded_preds[0]: .clearize:

 key text text into 
 العديد من الأواقب التي يمكنطلبى علىعاسة ال فيوف م مة من الهد الآخر. العلاقات الجنسرمة. هذه أناء العلاقاتقة والواصل بين العلاقات، خلال استخدامفاعل عنماع،تف فيتقترام الآكارك..اق التروج للعاًماع بالبعبةككما.. يمكن كنتاولتعك أنظهبار عنك ليس مجردريقة الوحيدة التي ت فيها استخدامقرب،، ف يكون إلى إلى تفكير في إذا كان هذه ال س الس تريدهم.اً إذا الجهمتع أن تتعلمب في شخصريكك فيكون ب بزيات م الجنسنسية، هذا هذاعب علىسي تعبية أونلا لائعة لظه النفسريكك ووفيف الضوتره. أنة إلىمارسة الجنس. يمكنحرص مع فيجانبك شكما و وماد إلى بعضكسائلائ لسق. بعضصة أو إذا أيضا أيضا ت بعضتب التيطرقوب. إذاساعد هذادركنسسما على نوعجة الاحةتكما إذا إذا كنت ترببي، الج العلاقةور، فهول ت تبدأقيدي عنليلاً إذا أيضالوس بعالاً جاك أو. بعضنايكدميكما.. إذا أيضا أيضادمة الك الكصص الممانسية أوكيضكما. خلال تجت اانتباه.ا عن الجنس.الغمما تتب تتمكنمن).). بعض قغككصةص). إذاحرصوا بعا علىنا إلى بعضصومات ابدأا عن أات مغنية التينعلبيهاقرب منكريكك.ني بتجكلوكما. ا، تنسلي ت من ذلكاههة بين شيءك.. تنتضانككما. اجعلسب فيملاعات الرياي وبعضكما.. تطلبي بعضعضا..عاقا. أجل.أدث عن عن الأالئلة حولفتعة لللاريكك"). ابدأا م ط واحد منعااطلين.ابست على جانبكما علىكابة على الأفس..احولا و اابتعل الأفسات.جرد أنانتهاء منكية الأود. ااروا بعض الأانتوه.فيفة.مشع من الأصفائ.لا الألم.. ا كنت تلقة بشأن تدي الردم، جنسسة،قيقيمةية، فع نفسسائد علىكما ش.مما الاقات ل الأتين. ا كنت تكنثر هذهسائد جبطانيات،قيفةتهاما، ف تأس. استخدامحبالهريكك عن كان يرغبوي أن ذلك آخر.خدمعي ذلكاومةته. إذا الجنس بطب الذي تكون فيه فيه. جانبضك إذانتي بعض الأشيواء والطانيتين.زرقة. اجعلان على مكانك.ابدي علىلمما. فينتئ في الأصص الرو ا ت هذالوسماقرب من بعضريكك إلى تعوبة في التاومة بعضج الأم.سب من  النتضانه إذا كنتاجت أنك تشعرك في تطوروز الحودك،،ثدة،هماقا، فحتعدي ق هذاريكك. تنتفي تلوس م مكان آخر. إذا إذا كنت تلقة بشأن تربة، شريكك،طرقناد عن عن، امكنكمافكع باستتك في أناح إلى نحو المثال إلىقولذ أن " "أروا معذيمني". أنلوس هنا". لكنعني نست في طرضكة".". أو "أعر بالحض التغ في أست بعضراحة فينجلس إلىطب". أو تساعدك هذاروج من مكانماكن الأخرى أو تسر الوتبكماية. إذا كنتحدثت ركما رور،درجةكعا للست بالر،سبالها، قد تأس من تحدث عنطلب "ك ت ترغبرين بالراحة. هذهمتار. إذا أن تبقىترم شريكك أنغباتك الجنس أساس. إذا إذا كنتاولتريكك إخاعك بأنضرورة حددود، تضعتها لك عامعمر، ف تحتاجين إلى التفكير في كيفيةغار العلاقةلاقتك.ه. إذا الرغم من أن قد الصمكن أن تجاعلك شريكك عن رغيه في واحدة، الأين،آخر، إلا أن من ي أن تصبح ش بالقلق حهدرك. إذاHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHumanHuman
decoded_refs[0]: في العلاقات السليمة، يمكنك بناء الثقة والتقارب من خلال التحدث والاستماع والمشاركة واحترام أفكاركما. يمكن استخدام نشاطات غير جنسية مثل اللعب بألعاب لوحية أو القراءة معا. يمكنك أيضا إجراء استطلاعات رأي أو مشاهدة فيلم مع وسائد وأضواء. إذا شعرت بالقلق، ابتعدي قليلا أو تقدمي لسببًا لنقل المكان. يجب أن يحترم شريكك رغباتك ويشعرك بالراحة.

In [5]:
from transformers import AutoTokenizer
import re
import random
from multiprocessing import cpu_count
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from evaluate import load as load_metric

In [6]:
model_id = "Qwen/Qwen2.5-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2048
  
# Set chat template
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

In [7]:
DEFAULT_CHAT_TEMPLATE

"{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"

In [8]:
raw_datasets = load_dataset("BounharAbdelaziz/Arabic-Synthetic-Summarization-Dataset-Filtered")

In [31]:
from datasets import load_dataset
from transformers import AutoTokenizer
import random
from multiprocessing import cpu_count

def create_conversation(example):
    """
    Transform the dataset into a conversational format.
    The user provides the text, and the assistant provides the summary.
    """
    # Create a conversation with user and assistant roles
    messages = [
        {"role": "user", "content": example["text"]},  # User provides the text
        {"role": "assistant", "content": example["summary"]}  # Assistant provides the summary
    ]
    # Return the conversation as a dictionary
    return {"messages": messages}

# Apply the chat template to the dataset
def apply_chat_template(example, tokenizer):
    example["text"] = tokenizer.apply_chat_template(example["messages"], tokenize=False)
    return example

dataset = load_dataset("BounharAbdelaziz/Arabic-Synthetic-Summarization-Dataset-Filtered")

# Load the model and tokenizer
model_id = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
    tokenizer.model_max_length = 2048

# Set chat template
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

# Load the dataset

# Transform the dataset into a conversational format
dataset = dataset.map(create_conversation, remove_columns=["text", "summary"])

dataset = dataset.map(
    apply_chat_template,
    num_proc=cpu_count(),
    fn_kwargs={"tokenizer": tokenizer},
    remove_columns=["messages"],
    desc="Applying chat template"
)

# Create the splits
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

# Print some samples to verify
num_samples_to_print = min(3, len(dataset["train"]))  # Ensure we don't exceed the dataset size
for index in random.sample(range(len(dataset["train"])), num_samples_to_print):
    print(f"Sample {index} of the processed training set:\n\n{dataset['train'][index]['text']}\n")

Applying chat template (num_proc=64): 100%|██████████| 444/444 [00:06<00:00, 65.22 examples/s]


Sample 1884 of the processed training set:

<|user|>
إن قام الطبيب بتشخيص حالتك بعدوى المتدثرة، سيقوم بوصف جرعة مضادات حيوية، وهي الطريقة الوحيدة لعلاج المرض إلى جانب تجنبه في المقام الأول. تختفي العدوى عادة خلال أسبوع أو أسبوعين.  خط العلاج الأول هو دواء azithromycin (1 جم كجرعة واحدة يوميا) أو doxycycline (100 مجم فمويا مرتين يوميا لمدة 7 أيام).  قد يتكون علاجك من جرعة تؤخذ مرة واحدة أو قد تحتاج إلى تناول العلاج عدة مرات يوميا لمدة 5-10 أيام.  يحتاج الشريك إلى العلاج أيضا حتى دون ظهور أعراض المتدثرة. سيؤدي ذلك إلى منعك أنت وشريكتك من نقل المرض بينكما مرة بعد أخرى.  لا تشارك أدوية علاج عدوى المتدثرة مع أي شخص آخر. إن كنت في مرحلة الحمل وكنت تعانين من عدوى المتدثرة، قد يصف لك الطبيب azithromycin في الثلث الثاني أو الثالث لتقليل خطر نقل المرض إلى الطفل. يتم علاج عدوى المتدثرة خلال الحمل في وقت اكتشافها، وسيتم اختبارها مجددا بعد ذلك للتأكد من زوال العدوى. بعد الولادة، سيقوم الطبيب بفحص المولود الجديد وعلاجه وفقا لذلك.  إن قمت بالولادة ونقلت عدوى المتدثرة إلى المولود الجديد، سيقوم الطبيب 

In [32]:
train_dataset

Dataset({
    features: ['summary_model_name', 'tokenizer_name', 'dataset_source', 'sequence_length', 'text'],
    num_rows: 3526
})

In [33]:
eval_dataset

Dataset({
    features: ['summary_model_name', 'tokenizer_name', 'dataset_source', 'sequence_length', 'text'],
    num_rows: 444
})

In [10]:
raw_datasets['train'] = raw_datasets['train'].select(range(2))
raw_datasets['validation'] = raw_datasets['validation'].select(range(2))
raw_datasets['test'] = raw_datasets['test'].select(range(2))

In [11]:
column_names = list(raw_datasets["train"].features)
raw_datasets = raw_datasets.map(apply_chat_template,
                                num_proc=cpu_count(),
                                fn_kwargs={"tokenizer": tokenizer},
                                remove_columns=column_names,
                                desc="Applying chat template",)

# create the splits
train_dataset = raw_datasets["train"]
eval_dataset = raw_datasets["test"]

for index in random.sample(range(len(raw_datasets["train"])), 3):
  print(f"Sample {index} of the processed training set:\n\n{raw_datasets['train'][index]['text']}")

num_proc must be <= 2. Reducing num_proc to 2 for dataset of size 2.
Applying chat template (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]


TypeError: string indices must be integers, not 'str'